# Exercise 1
**Making Publication Ready Drought Maps** 

In this exercise will be creating maps of drought events that can be used for report and scientific publications. For this exercise one can choose to use either select H SAF ASCAT 6.25 km Surface Soil Moisture (SSM) as monthly aggregated Z scores (January 1 2007 till May 31 2025) or dekadel Z scores (January 1 2011 till December 31 2019).

To create publication-ready plots we will be using the `matplotlib` library incombination with `cartopy`.

## Imports

In [ ]:
import cartopy.crs as ccrs
import geopandas as gpd
import hvplot.pandas  # noqa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rioxarray  # noqa
import xarray as xr
from ascat.utils import get_grid_gpis
from cartopy.mpl.ticker import (LatitudeFormatter, LatitudeLocator,
                                LongitudeFormatter, LongitudeLocator)
from fibgrid.realization import FibGrid
from pyresample import create_area_def, geometry
from pyresample.kd_tree import resample_gauss
from shapely.geometry import mapping

## Monthly Z scores

In [ ]:
%run ../src/download_path.py

url = make_url("ascat-6_25_ssm_monthly.csv")  # noqa
df_month = pd.read_csv(
    url,
    index_col=["time", "location_id"],
    parse_dates=["time"],
)

df_month.head()

## Monthly Z scores

In [ ]:
url = make_url("ascat-6_25-ssm-dekadal.csv")  # noqa
df_dekad = pd.read_csv(
    url,
    index_col=["time", "location_id"],
    parse_dates=["time"],
)

df_dekad.head()

## Selection of area of interest and time window

First we will select a time window:


In [ ]:
quicklook_args = dict(
    x="longitude",
    y="latitude",
    c="zscore",
    x_sampling=0.08,
    y_sampling=0.08,
    rasterize=True,
    crs=ccrs.PlateCarree(),
    tiles=True,
    cmap="reds_r",
    # clim=(-4, 0),
    frame_width=500,
    clabel="Drought anomaly",
)
df_dekad_20160121 = df_dekad.loc["2016-01-01"]
df_dekad_20160121.hvplot.points(**quicklook_args)

Then we select an area. Let's say a bound

In [ ]:
lon_min, lon_max = 32, 33
lat_min, lat_max = -26.5, -25.5

In [ ]:
fib = FibGrid(6.25, geodatum="WGS84")
gpis = get_grid_gpis(
    fib, bbox=[lat_min, lat_max, lon_min, lon_max]
)  # min long, max long
loc_mask = np.isin(df_dekad_20160121.index.get_level_values("location_id"), gpis)
df_dekad_20160121_maputo = df_dekad_20160121[loc_mask]
df_dekad_20160121_maputo.hvplot.points(**quicklook_args)



## Quick look

## Resample

For the publication ready plot we are going to resample the data to a regular grid, as opposed to quick viewing method.

In [ ]:
area_def = create_area_def(
    "maputo_area",
    {"proj": "longlat", "datum": "WGS84"},
    area_extent=[lon_min, lat_min, lon_max, lat_max],
    resolution=0.01,
    units="degrees",
    description="Global 0.01x0.01 degree lat-lon grid",
)

data = df_dekad_20160121_maputo.zscore.values
lons = df_dekad_20160121_maputo.longitude.values
lats = df_dekad_20160121_maputo.latitude.values


swath_def = geometry.SwathDefinition(lons=lons, lats=lats)
result = resample_gauss(
    swath_def, data, area_def, radius_of_influence=7500, sigmas=3750, fill_value=None
)
result

In [ ]:
lons = np.arange(32, 33, 0.01)
lats = np.flip(np.arange(-26.5, -25.5, 0.01))
da = xr.DataArray(
    result, dims=("latitude", "longitude"), coords={"latitude": lats, "longitude": lons}
).rio.write_crs(area_def.crs)

da

In [ ]:
country_border = gpd.read_file(make_url("country_border_10m.gpkg"))  # noqa


def clip_dataset_to_shape_file(ds, clip_shape):
    ds = ds.rio.clip(
        clip_shape.geometry.apply(mapping), clip_shape.crs, drop=False, invert=False
    )
    return ds


da_clipped = clip_dataset_to_shape_file(da, country_border).drop_vars("spatial_ref")
da_clipped

## Initial Plot

In [ ]:
da.plot()

## Basemap with Cartopy

First we tell Matplotlib which Reference System to use for Map Projection.

In [ ]:
projection = ccrs.Mercator()

Then we tell matplotlib what projection the data is in.

In [ ]:
crs = ccrs.PlateCarree()

We then formulate a canvas

In [ ]:
plt.figure(figsize=(8, 4.5), dpi=150)
ax = plt.axes(projection=projection, frameon=True)
gl = ax.gridlines(
    crs=crs,
    draw_labels=True,
    linewidth=0.6,
    color="gray",
    alpha=0.5,
    linestyle="-.",
)
gl.xlabel_style = {"size": 7}
gl.ylabel_style = {"size": 7}
ax.add_geometries(
    country_border.geometry,
    ccrs.PlateCarree(),
    facecolor="none",
    edgecolor="black",
    linewidth=2,
)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=crs)
fancy_datetime = pd.to_datetime("2016-01-01").strftime("%d %B %Y")
title = plt.title(f"Soil moisture anomaly over Maputo ({fancy_datetime})")

In [ ]:
plt.figure(figsize=(8, 8), dpi=150)
ax = plt.axes(projection=projection, frameon=True)
gl = ax.gridlines(
    crs=crs,
    draw_labels=True,
    linewidth=0.6,
    color="gray",
    alpha=0.5,
    linestyle="-.",
)
gl.xlabel_style = {"size": 7, "color": "gray"}
gl.ylabel_style = {"size": 7, "color": "gray"}

gl.top_labels, gl.right_labels = False, False

gl.ylocator = LatitudeLocator()
gl.xlocator = LongitudeLocator()
gl.xformatter = LongitudeFormatter()
gl.yformatter = LatitudeFormatter()

ax.add_geometries(
    country_border.geometry,
    ccrs.PlateCarree(),
    facecolor="none",
    edgecolor="black",
    linewidth=2,
)

vmin = -2.5
vmax = 0

levels = [-3, -2.5, -2, -1.5, -1, 0]
cbar_kwargs = {
    "orientation": "horizontal",
    "shrink": 0.6,
    "pad": 0.05,
    "aspect": 40,
    "label": "Drought Severity",
    "spacing": "proportional",
}
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=crs)

im = da_clipped.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="Reds_r",
    add_colorbar=False,
    levels=levels,
    vmin=vmin,
    vmax=vmax,
    alpha=0.8,
)


cbar = plt.colorbar(im, **cbar_kwargs)
cbar.set_ticks(levels)
cbar.set_ticklabels(["", "Extreme", "Severe", "Moderate", "Mild", "Normal"])
cbar.ax.tick_params(labelsize=9, rotation=45)  # Set custom labels


title = plt.title(f"Soil moisture anomaly over Maputo ({fancy_datetime})")